<a href="https://colab.research.google.com/github/hamzajg/movies-rec-colab/blob/master/MoviesRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%load_ext autoreload
%autoreload 2

In [6]:
!pip install scikit-surprise

     |████████████████████████████████| 6.5MB 3.0MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678215 sha256=3e165243f6d8a63a32c2f7fdf3b18866afa7746c7bcf6ccd677480446520fe6e
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [1]:
!git clone https://github.com/hamzajg/movies-rec-colab.git

Cloning into 'movies-rec-colab'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 59 (delta 22), reused 29 (delta 7), pack-reused 0
Unpacking objects: 100% (59/59), done.


In [0]:
%cd movies-rec-colab/

/content/movies-rec-colab


In [0]:
!git pull

remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 5), reused 5 (delta 2), pack-reused 0
Unpacking objects: 100% (11/11), done.
From https://github.com/hamzajg/movies-rec-colab
   4440613..0dc3bcb  master     -> origin/master
Updating 4440613..0dc3bcb
Fast-forward
 MoviesRec.ipynb       | 226 ++++++++++++++++++++++++++++++++++++++++++++++++++
 RecommenderMetrics.py | 154 ++++++++++++++++++++++++++++++++++
 2 files changed, 380 insertions(+)
 create mode 100644 MoviesRec.ipynb
 create mode 100644 RecommenderMetrics.py


In [13]:
%cd /content/

/content


In [7]:
import sys
sys.path.insert(0, 'movies-rec-colab')

from MovieLens import MovieLens

ml = MovieLens()
print('Loading Movies Rating..')
data = ml.loadMovieLensLatestSmall()

Loading Movies Rating..


In [8]:
print('Computing movie popularity ranks so we can measure novelty later..')
rankings = ml.getPopularityRanks()

Computing movie popularity ranks so we can measure novelty later..


In [9]:
from surprise import KNNBaseline
print('Computing item similarities so we can measure diversity later ...')
fullTrainSet = data.build_full_trainset()
sim_options = {'name': 'pearson_baseline', 'user_based': False}
simsAlgo = KNNBaseline(sim_options=sim_options)
simsAlgo.fit(fullTrainSet)

Computing item similarities so we can measure diversity later ...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [10]:
from surprise.model_selection import train_test_split
from surprise import SVD
print('Building recommendation models..')
trainSet, testSet = train_test_split(data, test_size=.25, random_state=1)
algo = SVD(random_state=10)
algo.fit(trainSet)

Building recommendation models..


In [14]:
print('Computing recommendations ..')
predictions = algo.test(testSet)
from RecommenderMetrics import RecommenderMetrics
print('Evaluating accuracy of model ..')
print('RMSE:', RecommenderMetrics.RMSE(predictions))
print('MAE:', RecommenderMetrics.MAE(predictions))

Computing recommendations ..
Evaluating accuracy of model ..
RMSE: 0.9033701087151801
MAE: 0.6977882196132263


In [22]:
print('Evaluating Top-10 Recommendation ..')
from surprise.model_selection import LeaveOneOut
# Set aside on rating per user for testing
LOOCV = LeaveOneOut(n_splits=1, random_state=1)

for trainSet, testSet in LOOCV.split(data):
  print('Computing recommmendation with leaveOneOut..')
  # Traim model without left-out ratings
  algo.fit(trainSet)
  # Predicts ratings fro left-out ratings only
  leftOutPredictions = algo.test(testSet)
  # Build predictions fro all ratings not in the training set
  print('Predict All Missing Ratings..')
  bigTestSet = trainSet.build_anti_testset()
  allPredictions = algo.test(bigTestSet)


Evaluating Top-10 Recommendation ..
Computing recommmendation with leaveOneOut..
Predict All Missing Ratings..
